<a href="https://colab.research.google.com/github/yashlal/Deepfake-Microbiomes/blob/main/Combination.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from newsolver import predict_community_fullnp
import numpy as np
import pandas as pd
import random as rd
from numba import njit
from numba.typed import List
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
import pickle
import torch.optim as optim
import time
from math import sqrt
import matplotlib.pyplot as plt

data = pd.read_excel('RealData.xlsx', index_col=0)
specs = data.columns.tolist()
trimmed_specs = []

for i in range(len(specs)):
    if data.iloc[:,i].astype(bool).sum() >= 85:
        trimmed_specs.append(specs[i])
dim1 = len(trimmed_specs)

typed_trimmed_specs = List()
[typed_trimmed_specs.append(x) for x in trimmed_specs]

@njit()
def get_LT(full_ar):
    ar = []
    for i in range(len(full_ar)):
        for j in range(i):
            ar.append(full_ar[i][j])
    return ar

@njit()
def generate_matrix(comm, tolerance):
    dim = len(comm)
    ar = np.zeros((dim,dim))

    for i in range(dim):
        for j in range(i+1):
            if i == j:
                ar[i][j] = 0
            else:
                r = rd.random()
                # m = mult[i*dim1+j]
                ar[i][j] = r
                ar[j][i] = (1-r)

    return ar

def datagen():
    lm = generate_matrix(typed_trimmed_specs, 0)
    cm = predict_community_fullnp(lm, trimmed_specs, verb=False)
    return (cm, get_LT(lm))

# select CUDA if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if str(device) == 'cuda:0':
	print('CUDA device selected!')
elif str(device) == 'cpu':
	print('CUDA device not available. CPU selected')

class MyNet(nn.Module):
    def __init__(self):
        super(MyNet, self).__init__()
        self.fc1 = nn.Linear(462, 462*5)
        self.fc2 = nn.Linear(462*5, 231*461)
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

net = MyNet().to(device)
criterion = nn.MSELoss(reduction='mean')
optimizer = optim.Adam(net.parameters(), lr=1e-3)

def test_net():
    test_loss = 0
    for i in range(100):
      x, y = datagen()
      test_x = torch.from_numpy(x).float().to(device)
      test_y = torch.FloatTensor(y).to(device)

      output = net(input).to(device)

      loss = criterion(output, true_y).to(device)
      test_loss += sqrt(loss.item())
    print(f'Average Test RMS: {test_loss/100}')

loss_v = []

for i in range(10000):
    if (i%1000==0) and (i!=0):
      test_net()
      continue

    optimizer.zero_grad()
    x, y = datagen()
    input = torch.from_numpy(x).float().to(device)
    true_y = torch.FloatTensor(y).to(device)

    output = net(input).to(device)

    loss = criterion(output, true_y).to(device)
    loss_v.append(sqrt(loss.item()))
    loss.backward()

    optimizer.step()

    print(f'Epoch {i}: {sqrt(loss.item())}')


PATH = 'model.pth'
torch.save(net.state_dict(), PATH)

plt.plot(loss_v)
plt.savefig('Loss.png')
plt.show()


CUDA device selected!
Epoch 0: 0.5778819939522026
Epoch 1: 0.554600695413029
Epoch 2: 0.5333948035380034
Epoch 3: 0.5113285120065738
Epoch 4: 0.4882696837009006
Epoch 5: 0.4629093279057983
Epoch 6: 0.43504537269035043
Epoch 7: 0.40840519918192314
Epoch 8: 0.38237542872733765
Epoch 9: 0.3565662544647929
Epoch 10: 0.3339296708842946
Epoch 11: 0.31518015720241754
Epoch 12: 0.3009237843454945
Epoch 13: 0.2934373795059168
Epoch 14: 0.2928324064248414
Epoch 15: 0.29671767734295534
Epoch 16: 0.3042847465673885
Epoch 17: 0.31215491314757454
Epoch 18: 0.3190968052676058
Epoch 19: 0.3247713641920667
Epoch 20: 0.32642153932838003
Epoch 21: 0.3258767877543598
Epoch 22: 0.322501148510152
Epoch 23: 0.31749740879247995
Epoch 24: 0.31208550859660034
Epoch 25: 0.3081760839437802
Epoch 26: 0.30334859496323024
Epoch 27: 0.3009579498655272
Epoch 28: 0.29898994402736373
Epoch 29: 0.2980883673187831
Epoch 30: 0.29792957222379757
Epoch 31: 0.2990783566224451
Epoch 32: 0.2980995521640581
Epoch 33: 0.302302797